In [46]:
import pandas as pd
import numpy as np

In [47]:
reading_history   = pd.read_csv("screening_databases/reading_history_database.csv")
assessment_scores = pd.read_csv("screening_databases/screening_assessment_scores.csv")
print("reading history matrix dimensions:   ", reading_history.shape)
print("assessment scores matrix dimensions: ", assessment_scores.shape)

reading history matrix dimensions:    (455, 34)
assessment scores matrix dimensions:  (453, 26)


In [48]:
# shorten the column names
shortened_cols = ["Q"+str(q) for q in range(1,34)]
shortened_cols.insert(0, "Participant")
reading_history.columns = shortened_cols

In [49]:
assessment_scores.rename(columns={ assessment_scores.columns[0]: "Participant" }, inplace=True)

In [50]:
# merge 2 databases on 'Participant' as key into one
merged_history_and_scores = pd.merge(reading_history, assessment_scores, how='outer', on='Participant')

In [51]:
# reset 'Participant' as the index
merged_history_and_scores.set_index('Participant', inplace=True)
print("merged history and scores dimensions: ", merged_history_and_scores.shape)

merged history and scores dimensions:  (488, 58)


In [52]:
# clean up to remove columns with text (Q30, and Q32)
#merged_history_and_scores.columns

In [53]:
del merged_history_and_scores['Q30']
del merged_history_and_scores['Q32']

In [54]:
#merged_history_and_scores.columns